In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [ ]:
df = pd.read_csv('/content/KDAG.csv')
df

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist


In [ ]:
df.iloc[0,6]

'3-Low'

In [ ]:
df['Physician_Segment'].unique()

array(['3-Low', '1-High', '2-Medium'], dtype=object)

In [ ]:
df = df[['Brand_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered', 'Market_Rx']]
df

,Brand_Rx,Sales_Rep_Calls,Samples_Dropped,Emails_Delivered,Market_Rx
0,0.0,1,0,0,2.80567
1,0.0,1,0,0,20.57312
2,0.0,1,0,0,6.16010
3,0.0,1,5,0,8.95501
4,0.0,1,0,0,9.13793
...,...,...,...,...,...
569995,0.0,0,0,1,2.45597
569996,0.0,0,0,0,2.38065
569997,0.0,0,0,1,7.42171
569998,0.0,0,0,1,5.39345


In [ ]:
def normalize(df):
    result = df.copy()
    for feature_name in df.columns:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result

In [ ]:
df = normalize(df)

In [ ]:
df

,Brand_Rx,Sales_Rep_Calls,Samples_Dropped,Emails_Delivered,Market_Rx
0,0.0,1.0,0.000000,0.000,0.010792
1,0.0,1.0,0.000000,0.000,0.079137
2,0.0,1.0,0.000000,0.000,0.023696
3,0.0,1.0,0.032051,0.000,0.034447
4,0.0,1.0,0.000000,0.000,0.035150
...,...,...,...,...,...
569995,0.0,0.0,0.000000,0.125,0.009447
569996,0.0,0.0,0.000000,0.000,0.009158
569997,0.0,0.0,0.000000,0.125,0.028549
569998,0.0,0.0,0.000000,0.125,0.020747


In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub

,Physician_ID,Channel
0,axt00001,1
1,axt00002,1
2,axt00003,1
3,axt00004,1
4,axt00005,1
...,...,...
9995,axt09996,1
9996,axt09997,1
9997,axt09998,1
9998,axt09999,1


In [ ]:
sub_brand=pd.read_csv('sample_brand.csv')
sub_brand

,Physician_ID,Expected_TRx
0,axt00001,0.0
1,axt00002,0.0
2,axt00003,0.0
3,axt00004,0.0
4,axt00005,0.0
...,...,...
9995,axt09996,0.0
9996,axt09997,0.0
9997,axt09998,0.0
9998,axt09999,0.0


In [ ]:
for k in range(10000):
  if(k % 1000 == 0):
    print(k)
  start = k * 57
  data = df.iloc[start:start + 57,:]
  # display(data)
  data = data[['Brand_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered', 'Market_Rx']]

  train = data[:int(0.8*(len(data)))]
  valid = data[int(0.8*(len(data))):]

  #fit the model
  from statsmodels.tsa.vector_ar.var_model import VAR

  model = VAR(endog=train)
  model_fit = model.fit(trend='nc')

  # make prediction on validation
  prediction = model_fit.forecast(model_fit.y, steps=len(valid))

  model = VAR(endog=data)
  model_fit = model.fit(trend='nc')
  yhat = model_fit.forecast(model_fit.y, steps=1)

  call = data.iloc[4:,1].sum()

  if(yhat[0][0] != 0):
    sub_brand.iloc[k,1] = yhat[0][0]

  else:
    if df.iloc[start,6] == '3-Low':
      sub_brand.iloc[k,1] = 0.245 / 2
    elif df.iloc[start,6] == '2-Medium':
      sub_brand.iloc[k,1] = 0.520 / 2
    else:
      sub_brand.iloc[k,1] = 1.770 / 2

  if(yhat[0][1] >= yhat[0][2] and yhat[0][1] >= yhat[0][3]):
    if df.iloc[start,6] == '3-Low' and call > 12:
      sub.iloc[k,1] = 1
    elif df.iloc[start,6] == '2-Medium' and call > 24:
      sub.iloc[k,1] = 1
    elif df.iloc[start,6] == '1-High' and call > 48:
      sub.iloc[k,1] = 1
    else:
      sub.iloc[k,1] = 0

  elif(yhat[0][2] >= yhat[0][3]):
    sub.iloc[k,1] = 1

  else:
    sub.iloc[k,1] = 2

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
df = pd.read_csv('HCP_Data_KDAG_Hackathon.csv')
df

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist


In [ ]:
df.tail()

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist
569999,axt10000,20200131,0.0,0.00000,0,0,3-Low,0,0,0,Dermatologist


In [ ]:
semi=np.zeros((10000,6))
semi

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
#harsh..... with constraints


for k in range(10000):
  if(k % 1000 == 0):
    print(k)
  start = k * 57
  data = df.iloc[start:start + 58,:]
  # display(data)
  #data = data[['Brand_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered', 'Market_Rx']]
  data = data[['Brand_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered']]

  train = data[:int(0.8*(len(data)))]
  valid = data[int(0.8*(len(data))):]

  #fit the model
  from statsmodels.tsa.vector_ar.var_model import VAR

  model = VAR(endog=train)
  model_fit = model.fit(trend='nc')

  # make prediction on validation
  prediction = model_fit.forecast(model_fit.y, steps=len(valid))

  model = VAR(endog=data)
  model_fit = model.fit(trend='nc')
  yhat = model_fit.forecast(model_fit.y, steps=2)

  call = data.iloc[5:,1].sum()

  semi[k][0]=yhat[0][1]+yhat[0][3]-2*yhat[0][2]    # call+mail-2*drops
  semi[k][1]=yhat[0][1]  #calls
  semi[k][2]=yhat[0][2]  #drops
  semi[k][3]=yhat[0][3]  #mails
  semi[k][4]=call
  semi[k][5]=k

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
semi_sorted=semi[semi[:,0].argsort()]
semi_sorted
semi=semi_sorted

In [ ]:
count_drop=0
max_drop=2500
for k in range(10000):
  if(k % 1000 == 0):
    print(k)
  if(semi[k][2]>semi[k][1] and semi[k][2]>semi[k][3]):
    count_drop=count_drop+1
diff=max_drop-count_drop
print(diff)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
976


In [ ]:
sub = pd.read_csv('sample_submission.csv')
sub

,Physician_ID,Channel
0,axt00001,1
1,axt00002,1
2,axt00003,1
3,axt00004,1
4,axt00005,1
...,...,...
9995,axt09996,1
9996,axt09997,1
9997,axt09998,1
9998,axt09999,1


In [ ]:
count=0
blah=0
for k in range(10000):
  #start = k * 57
  ind=int(semi[k][5])
  if(k % 1000 == 0):
    print(k)
  if(semi[k][2] > semi[k][1] and semi[k][2] >= semi[k][3] and count<2500):
    sub.iloc[ind,1] = 1
    count+=1
  elif(semi[k][1] >= semi[k][3]):
    if df.iloc[start,6] == '3-Low' and semi[k][4] <= 12:
      sub.iloc[ind,1] = 0
    elif df.iloc[start,6] == '2-Medium' and semi[k][4] <= 24:
      sub.iloc[ind,1] = 0
    elif df.iloc[start,6] == '1-High' and semi[k][4] <= 48:
      sub.iloc[ind,1] = 0
    else:
      if(semi[k][2] >= semi[k][3] and count<2500):
        sub.iloc[ind,1] = 1
        count+=1
      else:
        sub.iloc[ind,1] = 2
  elif(semi[k][2] >= semi[k][3] and count<2500):
    sub.iloc[ind,1] = 1
    count+=1
  else:
    sub.iloc[ind,1] = 2

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
count=0
for k in range(10000):
  ind=int(semi[k][5])
  start = k * 57
  if(k % 1000 == 0):
    print(k)
  if(semi[k][1] >= semi[k][2] and semi[k][1] >= semi[k][3]):
    if df.iloc[start,6] == '3-Low' and semi[k][4] >= 10:
      sub.iloc[ind,1] = 2
    elif df.iloc[start,6] == '2-Medium' and semi[k][4] >= 10:
      sub.iloc[ind,1] = 2
    elif df.iloc[start,6] == '1-High' and semi[k][4] >= 10:
      sub.iloc[ind,1] = 2
    else:
      sub.iloc[ind,1] = 0

  elif(semi[k][2] > semi[k][3] and count<2500):
    sub.iloc[ind,1] = 1
    count+=1

  else:
    sub.iloc[ind,1] = 2

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
count=0
for k in range(9999,-1,-1):
  if(k % 1000 == 0):
    print(k)
  if(semi[k][1] >= semi[k][2] and semi[k][1] >= semi[k][3]):
    if df.iloc[start,6] == '3-Low' and semi[k][4] >= 12:
      sub.iloc[k,1] = 2
    elif df.iloc[start,6] == '2-Medium' and semi[k][4] >= 24:
      sub.iloc[k,1] = 2
    elif df.iloc[start,6] == '1-High' and semi[k][4] >= 48:
      sub.iloc[k,1] = 2
    else:
      sub.iloc[k,1] = 0

  elif(semi[k][2] > semi[k][3] and count<2500):
    sub.iloc[k,1] = 1
    count+=1

  else:
    sub.iloc[k,1] = 2

9000
8000
7000
6000
5000
4000
3000
2000
1000
0


In [ ]:
sub

,Physician_ID,Channel
0,axt00001,1
1,axt00002,0
2,axt00003,0
3,axt00004,2
4,axt00005,1
...,...,...
9995,axt09996,0
9996,axt09997,0
9997,axt09998,2
9998,axt09999,2


In [ ]:
sub['Channel'].value_counts()

2    4331
0    3169
1    2500
Name: Channel, dtype: int64

In [ ]:
sub.to_csv('hs_2.csv', index = False)

In [ ]:
sub_brand

,Physician_ID,Expected_TRx
0,axt00001,0.498871
1,axt00002,0.245000
2,axt00003,0.245000
3,axt00004,0.245000
4,axt00005,0.716437
...,...,...
9995,axt09996,0.245000
9996,axt09997,0.520000
9997,axt09998,0.245000
9998,axt09999,0.453266


In [ ]:
sub_brand.to_csv('ts_brand.csv', index = False)

In [ ]:
sub_brand['Expected_TRx'].max()

19.92354258419902

In [ ]:
from numpy import array

In [ ]:
data = df.iloc[:57,:]

In [ ]:
data = data[['Brand_Rx', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered']]

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = data['Brand_Rx'].values
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[0. 0. 0.] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.841
[0.    0.    0.841] 0.0
[0.    0.841 0.   ] 0.0
[0.841 0.    0.   ] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.842
[0.    0.    0.842] 0.0
[0.    0.842 0.   ] 0.0
[0.842 0.    0.   ] 0.899
[0.    0.    0.899] 0.0
[0.    0.899 0.   ] 0.0
[0.899 0.    0.   ] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 0.0
[0. 0. 0.] 1.08
[0.   0.   1.08] 1.075
[0.    1.08  1.075] 0.0
[1.08  1.075 0.   ] 3.181
[1.075 0.    3.181] 0.0
[0.    3.181 0.   ] 1.08
[3.181 0.    1.08 ] 0.931
[0.    1.08  0.931] 0.0
[1.08  0.931 0.   ] 1.022
[0.931 0.    1.022] 2.166
[0.    1.022 2.166] 1.878
[1.022 2.166 1.878] 0.0
[2.166 1.878 0.   ] 0.0
[1.878 0.    0.   ] 2.096
[0.    0.    2.096] 2.196
[0.    2.096 2.196] 1.02
[2.096 2.196 1.02 ] 1.024
[2.196 1.02  1.024] 0.884
[1.02  1.024 0.884] 0.0
[1.024 0.884 0.   ] 0.0
[0.884 0.    0.   ] 3.111
[0.    0.    3.111] 1.015
[0.    3.111 1.015] 0.844
[3.111 1.015 0.844] 0.0
[1.015 0.844 0.   ] 1.016
[0.844 0.    1.016] 1.02
[0.    1.016 1.02 ] 0.861
[1.

In [ ]:
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

for j in range(4):
  # define input sequence
  raw_seq = data.iloc[:,j].values
  # choose a number of time steps
  n_steps = 3
  # split into samples
  X, y = split_sequence(raw_seq, n_steps)
  # reshape from [samples, timesteps] into [samples, timesteps, features]
  n_features = 1
  X = X.reshape((X.shape[0], X.shape[1], n_features))
  # define model
  model = Sequential()
  model.add(LSTM(50, activation='relu', input_shape=(n_steps, n_features)))
  model.add(Dense(1))
  model.compile(optimizer='adam', loss='mse')
  # fit model
  model.fit(X, y, epochs=200, verbose=0)
  # demonstrate prediction
  x_input = data.iloc[54:,j].values
  x_input = x_input.reshape((1, n_steps, n_features))
  yhat = model.predict(x_input, verbose=0)
  print(yhat)

[[0.7835244]]


In [ ]:
data.loc[54:,'Brand_Rx'].values

array([0.847, 0.   , 1.027])

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from sklearn.metrics import mean_squared_error

In [ ]:
train = data[:int(0.8*(len(data)))]
valid = data[int(0.8*(len(data))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

In [ ]:
cols = data.columns

#converting predictions to dataframe
pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])
for j in range(0,4):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check mse
for i in cols:
    print('mse value for', i, 'is : ', mean_squared_error(pred[i], valid[i]))

mse value for Brand_Rx is :  0.40863335292144937
mse value for Sales_Rep_Calls is :  0.28122386496709134
mse value for Samples_Dropped is :  2.799691796641604
mse value for Emails_Delivered is :  0.30029858535555354


In [ ]:
model = VAR(endog=data)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=1)
print(yhat)

[[0.65250689 0.7193596  1.03744729 0.21026292]]


In [ ]:
yhat[0][0]

0.6525068948690246

In [ ]:
df_new = df
df_new['Sales_Rep_Calls'] = (df_new['Sales_Rep_Calls'] > 0).astype(int)
df_new['Samples_Dropped'] = (df_new['Samples_Dropped'] > 0).astype(int)
df_new['Emails_Delivered'] = (df_new['Emails_Delivered'] > 0).astype(int)

In [ ]:
df = df_new

In [ ]:
sub = pd.read_csv('/content/Sample.csv')
sub

,Physician_ID,Channel
0,axt00001,1
1,axt00002,1
2,axt00003,1
3,axt00004,1
4,axt00005,1
...,...,...
9995,axt09996,1
9996,axt09997,1
9997,axt09998,1
9998,axt09999,1


In [ ]:
df1 = df[df.index % 57 == 4]
df1

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
4,axt00001,20190201,0.000,9.13793,1,0,3-Low,0,0,0,Dermatologist
61,axt00002,20190201,0.000,3.19303,0,0,3-Low,0,0,0,Dermatologist
118,axt00003,20190201,1.054,7.54276,1,0,3-Low,0,0,0,Dermatologist
175,axt00004,20190201,0.000,0.00000,0,0,3-Low,0,0,0,Dermatologist
232,axt00005,20190201,0.000,6.59857,0,0,1-High,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569719,axt09996,20190201,0.000,13.23480,0,0,3-Low,0,0,0,Dermatologist
569776,axt09997,20190201,1.040,2.79760,0,0,2-Medium,0,0,0,Nurse Practitioner
569833,axt09998,20190201,0.000,2.36451,0,0,3-Low,0,0,0,Dermatologist
569890,axt09999,20190201,0.000,5.29123,1,0,1-High,0,0,0,Dermatologist


In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 569943
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Physician_ID               10000 non-null  object 
 1   Time_Period                10000 non-null  int64  
 2   Brand_Rx                   10000 non-null  float64
 3   Market_Rx                  10000 non-null  float64
 4   Sales_Rep_Calls            10000 non-null  int64  
 5   Samples_Dropped            10000 non-null  int64  
 6   Physician_Segment          10000 non-null  object 
 7   Emails_Delivered           10000 non-null  int64  
 8   Speaker_Programs_Attended  10000 non-null  int64  
 9   Vouchers_Dropped           10000 non-null  int64  
 10  Specialty                  10000 non-null  object 
dtypes: float64(2), int64(6), object(3)
memory usage: 937.5+ KB


In [ ]:
for i in range(len(df1)):
  if df1.iloc[i,4] == 0 and df1.iloc[i,7] == 0 and df1.iloc[i,5] == 0:
    sub.iloc[i,1] = np.random.choice(3,1,p=[0.6, 0.2, 0.2])
  elif df1.iloc[i,4] >= df1.iloc[i,7] and df1.iloc[i,4] >= df1.iloc[i,5]:
    sub.iloc[i,1] = 0
  # elif df1.iloc[i,7] >= df1.iloc[i,5]:
  #   sub.iloc[i,1] = 2
  else:
    sub.iloc[i,1] = np.random.choice([1,2],1)

In [ ]:
sub['Channel'].value_counts()

0    6569
1    1728
2    1703
Name: Channel, dtype: int64

In [ ]:
sub.to_csv('sub_feb.csv', index = False)

In [ ]:
s = pd.read_csv('/content/sample_submission.csv')
s

,Physician_ID,Expected_TRx
0,axt00001,0.0
1,axt00002,0.0
2,axt00003,0.0
3,axt00004,0.0
4,axt00005,0.0
...,...,...
9995,axt09996,0.0
9996,axt09997,0.0
9997,axt09998,0.0
9998,axt09999,0.0


In [ ]:
for i in range(len(df1)):
  s.iloc[i,1] = df1.iloc[i,2]

In [ ]:
s

,Physician_ID,Expected_TRx
0,axt00001,0.000
1,axt00002,0.000
2,axt00003,0.000
3,axt00004,0.000
4,axt00005,0.000
...,...,...
9995,axt09996,0.000
9996,axt09997,1.034
9997,axt09998,0.000
9998,axt09999,0.000


**##a1b1+a2b2+a3b3 approach##**


In [ ]:
import pandas as pd
import numpy as np
from pandas import read_excel

In [ ]:
df = pd.read_csv('HCP_Data_KDAG_Hackathon.csv')
df

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,5,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist


In [ ]:
df_new = df
df_new['Sales_Rep_Calls'] = (df_new['Sales_Rep_Calls'] > 0).astype(int)
df_new['Samples_Dropped'] = (df_new['Samples_Dropped'] > 0).astype(int)
df_new['Emails_Delivered'] = (df_new['Emails_Delivered'] > 0).astype(int)


In [ ]:
df=df_new
df

,Physician_ID,Time_Period,Brand_Rx,Market_Rx,Sales_Rep_Calls,Samples_Dropped,Physician_Segment,Emails_Delivered,Speaker_Programs_Attended,Vouchers_Dropped,Specialty
0,axt00001,20190104,0.0,2.80567,1,0,3-Low,0,0,0,Dermatologist
1,axt00001,20190111,0.0,20.57312,1,0,3-Low,0,0,0,Dermatologist
2,axt00001,20190118,0.0,6.16010,1,0,3-Low,0,0,0,Dermatologist
3,axt00001,20190125,0.0,8.95501,1,1,3-Low,0,0,0,Dermatologist
4,axt00001,20190201,0.0,9.13793,1,0,3-Low,0,0,0,Dermatologist
...,...,...,...,...,...,...,...,...,...,...,...
569995,axt10000,20200103,0.0,2.45597,0,0,3-Low,1,0,0,Dermatologist
569996,axt10000,20200110,0.0,2.38065,0,0,3-Low,0,0,0,Dermatologist
569997,axt10000,20200117,0.0,7.42171,0,0,3-Low,1,0,0,Dermatologist
569998,axt10000,20200124,0.0,5.39345,0,0,3-Low,1,0,0,Dermatologist


In [ ]:
sub = pd.read_csv('sample_submission.csv')

In [ ]:
sub_brand=pd.read_csv('sample_brand.csv')

In [ ]:
for k in range(10000):
  if(k%1000==0):
    print(k)
  start=k*57
  sum=0.0
  count=0
  for j in range(4,57):
    if(start+j<=len(df)):
      value=df.iloc[start+j,2]
      if(value>0):
        sum=sum+value
        count=count+1

  if(count):
    avg=sum/53
    sub_brand.iloc[k,1]=1* avg
    """
  else:
    if df.iloc[start,6] == '3-Low':
      sub_brand.iloc[k,1] = 0.245 / 2
    elif df.iloc[start,6] == '2-Medium':
      sub_brand.iloc[k,1] = 0.520 / 2
    else:
      sub_brand.iloc[k,1] = 1.770 / 2
      """


0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
sub_brand.to_csv('brandavg_1.csv',index=False)

In [ ]:
sub['Channel'].value_counts()

0    6649
2    3054
1     297
Name: Channel, dtype: int64

In [ ]:
sub

,Physician_ID,Channel
0,axt00001,1
1,axt00002,0
2,axt00003,2
3,axt00004,0
4,axt00005,0
...,...,...
9995,axt09996,0
9996,axt09997,0
9997,axt09998,0
9998,axt09999,2


In [ ]:
sub_brand

,Physician_ID,Expected_TRx
0,axt00001,0.629379
1,axt00002,0.000000
2,axt00003,0.515929
3,axt00004,0.035232
4,axt00005,0.650328
...,...,...
9995,axt09996,0.000000
9996,axt09997,0.054637
9997,axt09998,0.000000
9998,axt09999,0.887369


In [ ]:
sub.to_csv('sub_weight_6.csv',index=False)

In [ ]:
for k in range(10000):
  if(k%1000==0):
    print(k)
  start=k*57
  f_call,f_drop,f_mail=0,0,0
  score_call,score_drop,score_mail=0.0,0.0,0.0
  for j in range(57):
    if(start+j<=len(df)):
      value=df.iloc[start+j,2]
      tot_value=df.iloc[start+j,3]

      f1=df.iloc[start+j,4]
      f2=df.iloc[start+j,5]
      f3=df.iloc[start+j,7]

      sum=0
      sum=f1+f2+f3
      f_call=f_call+f1
      f_drop=f_drop+f2
      f_mail=f_mail+f3
      frac=0.0
      if(sum>0 and tot_value>0):
        frac=value/tot_value
        score_call=score_call+f_call*frac/sum
        score_drop=score_drop+f_drop*frac/sum
        score_mail=score_mail+f_mail*frac/sum

  #AS denotes average score
  AS_call,AS_drop,AS_mail=0.0,0.0,0.0
  if(f_call):
    AS_call=score_call/f_call
  if(f_drop):
    AS_drop=score_drop/f_drop
  if(f_mail):
    AS_mail=score_mail/f_mail

  if(AS_call>=AS_drop and AS_call>=AS_mail):
    sub.iloc[k,1] = 0
    sub_brand.iloc[k,1]= AS_call
  elif(AS_drop>=AS_mail):
    sub.iloc[k,1] = 1
    sub_brand.iloc[k,1]= AS_drop
  else:
    sub.iloc[k,1] = 2
    sub_brand.iloc[k,1]= AS_mail

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
  n_vars = 1 if type(data) is list else data.shape[1]
  df = DataFrame(data)
  cols, names = list(), list()
  # input sequence (t-n, ... t-1)
  for i in range(n_in, 0, -1):
  cols.append(df.shift(i))
  names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
  # forecast sequence (t, t+1, ... t+n)
  for i in range(0, n_out):
  cols.append(df.shift(-i))
  if i == 0:
  names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
  else:
  names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
  # put it all together
  agg = concat(cols, axis=1)
  agg.columns = names
  # drop rows with NaN values
  if dropnan:
  agg.dropna(inplace=True)
  return agg

In [ ]:
dataset = read_csv('pollution.csv', header=0, index_col=0)
values = dataset.values
# integer encode direction
encoder = LabelEncoder()
values[:,4] = encoder.fit_transform(values[:,4])
# ensure all data is float
values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)
# drop columns we don't want to predict
reframed.drop(reframed.columns[[9,10,11,12,13,14,15]], axis=1, inplace=True)
print(reframed.head())